# Introduction

This guide will show you how to:

* Install ```neptune-client```,
* Connect Neptune to your Colab notebook and create the first run,
* Log simple metrics to Neptune and explore them in the UI.

# Step 0: Before you start

Make sure that:
* you have an account with both [Google](https://support.google.com/accounts/answer/27441?hl=en) and [Neptune](https://neptune.ai/register)
* you have [created a project](https://app.gitbook.com/@neptune-ai/s/docs-re-positioning/administration/workspace-project-and-user-management/projects#create-project) from the Neptune UI that you will use for tracking metadata.

**Tip:**<br>
Registering with Neptune and creating a project is optional in case you are just trying out the application as an 'ANONYMOUS' user.

# Step 1: Install ```neptune-client``` and import `neptune`

In [1]:
! pip install --upgrade --quiet neptune-client==0.9.13

In [3]:
import neptune.new as neptune

# Step 2: Initialize Neptune

Connect your script to Neptune and create a new run.

Basically, you tell Neptune: 

* **who you are**: your Neptune `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point, you will have a new run in Neptune. From now on you will use `run` to log metadata to it.

### Get your personal ```api_token``` to initialize Neptune

**Note:**<br>
There are a few special, public projects to show how Neptune works. For those projects, you can use the 'ANONYMOUS' api token and log as a public user `neptuner`.  

For example:  
```python
run = neptune.init(api_token='ANONYMOUS',
                   project='common/neptune-and-google-colab')
```

Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#authentication) and pass it to Neptune:
![get_token.gif](https://gblobscdn.gitbook.com/assets%2F-MXTao65xpmg6bawJ5vk%2F-MYZt9HJXRALUZWcKCSo%2F-MY_02rjQ90_XiRgTJ6s%2Fget_token.gif?alt=media&token=3de485e3-5ee2-4b98-b991-33a5b70086b5)

The preferred way of doing this is by using the ```getpass()``` method so that your token remains private even if you share the notebook.

In [4]:
from getpass import getpass
api_token = getpass('Enter your private Neptune API token: ')

# you can log as an anonymous user neptune with api_token='ANONYMOUS'
# api_token = 'ANONYMOUS' 

### Initialize your project

Remember to [create a new project](https://docs.neptune.ai/administration/workspace-project-and-user-management/projects#create-project) from the UI that you will use for metadata tracking.

In [5]:
workspace = 'your_neptune_username'
project_name = 'neptune-and-google-colab'
project = workspace + '/' + project_name

# if you are using ANONYMOUS api token log to the project= 'common/neptune-and-google-colab'
# project = 'common/neptune-and-google-colab'

print(project)

common/neptune-and-google-colab


**Tip:** The `project_qualified_name` of a project can be found under its Settings → Properties
![](https://gblobscdn.gitbook.com/assets%2F-MXTao65xpmg6bawJ5vk%2F-MYjnZwuxt-ftuLX0jLn%2F-MYjqWqsqrA-Z95dn0Vx%2FScreenshot_2021-04-20%20COLAB%20basic-colab.png?alt=media&token=506bce0d-8ebc-472b-9a4e-63a8aeeca172)

In [6]:
run = neptune.init(project=project, api_token=api_token)

Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://ui.neptune.ai/common/neptune-and-google-colab/e/NEP-7


Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next. 

Runs can be viewed as dictionary-like structures - **namespaces** - that you can define in your code. You can apply a hierarchical structure to your metadata that will be reflected in the UI as well. Thanks to this you can easily organize your metadata in a way you feel is most convenient.

# Step 3: Log metadata during training

Log metrics or losses under a name of your choice. You can log one or multiple values.

Now run the cell below, and switch over to the Neptune UI to view the live logging.

In [7]:
from time import sleep

params = {'learning_rate': 0.1}

# log params
run['parameters'] = params

# log name and append tags
run["sys/name"] = 'colab-example'
run["sys/tags"].add(['colab', 'simple'])

# log loss during training
for epoch in range(132):
    sleep(0.1) # to see logging live
    run["train/loss"].log(0.97 ** epoch)
    run["train/loss-pow-2"].log((0.97 ** epoch)**2)

# log train and validation scores
run['train/accuracy'] = 0.95
run['valid/accuracy'] = 0.93

# log files/artifacts
! echo "Welcome to Neptune" > file.txt
run['artifacts/sample'].upload('file.txt') # file will be uploaded as sample.txt

The snippet above logs:

* `parameters` with just one field: learning rate,
* name of run and two tags,
* `train/loss` and `train/loss-pow-2` as series of numbers, visualized as charts in UI,
* `train/accuracy` and `valid/accuracy` as single values
* `file.txt` which will be visible under All Metadata/artifacts as sample.txt

![upload_file_as_artifact.png](https://gblobscdn.gitbook.com/assets%2F-MXTao65xpmg6bawJ5vk%2F-MYZt9HJXRALUZWcKCSo%2F-MY_5xqm2_MtE0usG2bV%2Fupload_file_as_artifact.png?alt=media&token=d861bd06-6389-475f-99fb-b34ec4bc3d06)

Go to the `All metadata` and `Charts` sections of the Neptune UI to see them. You can also check an [example run](https://app.neptune.ai/o/common/org/showroom/e/SHOW-37/charts) 

![colab_example_charts.png](https://gblobscdn.gitbook.com/assets%2F-MXTao65xpmg6bawJ5vk%2F-MYZt9HJXRALUZWcKCSo%2F-MY_69KYOwzBEXczhIss%2Fcolab_example_charts.png?alt=media&token=3ffdc950-581d-4d44-8de3-bce0d4493f5c)

**Tip:**

Neptune automatically logs the hardware consumption during the run. 

You can see it in the `Monitoring` section of the Neptune UI. 

![image](https://neptune.ai/wp-content/uploads/Product_hardware-usage-1.png)

**Tip:**<br>
To view the structure of a run, use the `print_structure()` method.

In [8]:
run.print_structure()

'artifacts':
    'sample': File
'monitoring':
    'cpu': FloatSeries
    'memory': FloatSeries
    'stderr': StringSeries
    'stdout': StringSeries
'parameters':
    'learning_rate': Float
'source_code':
    'git': GitRef
'sys':
    'creation_time': Datetime
    'description': String
    'hostname': String
    'id': String
    'modification_time': Datetime
    'name': String
    'owner': String
    'ping_time': Datetime
    'running_time': Float
    'size': Float
    'state': RunState
    'tags': StringSet
'train':
    'accuracy': Float
    'loss': FloatSeries
    'loss-pow-2': FloatSeries
'valid':
    'accuracy': Float


# Step 4: Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [9]:
run.stop()

# Conclusion

You’ve learned how to:
* Install `neptune-client`,
* Connect Neptune to your Google Colab notebook and create a run,
* Log metadata to Neptune,
* See your metrics parameters and scores,
* See hardware consumption during the run.

# What's next

Now that you know how to create runs and log metrics, you can learn:

* [How to log other types of metadata to Neptune](https://docs.neptune.ai/you-should-know/logging-and-managing-runs-results/logging-runs-data#what-objects-can-you-log-to-neptune)
* [How to download runs data from Neptune](https://docs.neptune.ai/user-guides/logging-and-managing-runs-results/downloading-runs-data)
* [How to connect Neptune to the ML framework you are using](https://docs.neptune.ai/essentials/integrations)